In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from googletrans import Translator
from tqdm import tqdm

In [9]:
sheet_name = pd.ExcelFile('2.xlsx').sheet_names
sheet_names = []
# Print the sheet names
for s in sheet_name:
    sheet_names.append(s)
sheet_names

['مشاغل مدیریت',
 'مشاغل تجاری و فعالیت های مالی',
 'مشاغل کامپیوتر و ریاضی',
 'مشاغل معماری و مهندسی',
 'مشاغل دفتر پشتیبانی و اداری',
 'مشاغل تولید',
 'مشاغل زندگی، فیزیکی وعلوم اجتما',
 'مشاغل اجتماعی وخدمات اجتماعی',
 'مشاغل حقوقی',
 'مشاغل آموزش وپرورش،آموزش،کتابخا',
 'مشاغل هنر،سرگرمی ورزشی ورسانه ه',
 'مشاغل بهداشت ودرمان پزشکی']

In [10]:
dataframes = pd.read_excel("2.xlsx", sheet_name=None)
for sheet_name, df in dataframes.items():
    print("Sheet:", sheet_name)
    print(df)

Sheet: مشاغل مدیریت
    مشاغل مدیریت                                        مدیران عالی  \
0            NaN                                                NaN   
1            NaN                                                NaN   
2            NaN                                                NaN   
3            NaN  تبلیغات ، بازاریابی ، ترفیعات ، روابط عمومی و ...   
4            NaN                                                NaN   
5            NaN                                                NaN   
6            NaN                                                NaN   
7            NaN                                                NaN   
8            NaN                             مدیران فعالیتهای تخصصی   
9            NaN                                                NaN   
10           NaN                                                NaN   
11           NaN                                                NaN   
12           NaN                                         

In [11]:
for i in sheet_names:
    dataframes[i]['job_title_farsi'] = dataframes[i].iloc[:,3]

In [12]:
translator = Translator()

def translate_job_title(row):
    if row['job_title_farsi'] is not None:
        try:
            translation = translator.translate(row['job_title_farsi'], src='fa', dest='en')
            if translation is not None and translation.text is not None:
                return translation.text
        except Exception:
            pass
    return ''
for i in sheet_names:
     dataframes[i]['job_title_English'] = dataframes[i].apply(translate_job_title, axis=1)

In [13]:
for i in sheet_names:
    dataframes[i]['url'] = "https://www.onetonline.org/find/result?s=" + dataframes[i]['job_title_English']
    for index, url in enumerate(dataframes[i]['url']):
        result = requests.get(url)
        soup = BeautifulSoup(result.text, 'html.parser')
        code = soup.find('td', {'class': 'text-end w-15 mw-8e'})
        dataframes[i].loc[index, 'code'] = code.text if code else ''

In [14]:
for i in sheet_names:
    dataframes[i]['url2'] = "https://www.onetonline.org/link/summary/"+dataframes[i]['code']

In [15]:
writer = pd.ExcelWriter('total.xlsx')

for sheet_name in dataframes.keys():
    Job_Titles = []
    Job_Descriptions = []
    Tasks = []
    Skills = []
    Knowledges = []
    Technology_Skills = []
    
    for u in dataframes[sheet_name]['url2']:
        result = requests.get(u)
        soup = BeautifulSoup(result.text, 'html.parser')
        
        Job_Title = soup.find('span', {'class': 'main'})
        if Job_Title is not None:
            Job_Titles.append(Job_Title.text)
        else:
            Job_Titles.append('')
        
        Job_Description = soup.find('p')
        if Job_Description is not None:
            Job_Descriptions.append(Job_Description.text)
        else:
            Job_Descriptions.append('')
        
        Task = soup.find('ul', {'class': 'list-unstyled m-0'})
        if Task is not None:
            task_text = Task.text.replace('\nRelated occupations\n\n\n\n\n\n', '\n').replace('\nRelated occupations\n\n\n\n', '\n').replace('\n\n', '')
            Tasks.append(task_text)
        else:
            Tasks.append('')
        
        Skill = soup.find('div', {'id': 'Skills'})
        if Skill is not None:
            ul = Skill.find('ul', {'class': 'list-unstyled m-0'})
            if ul is not None:
                skill_text = ul.text.replace('\nRelated occupations\n\n\n\n\n\n', '\n').replace('\nRelated occupations\n\n\n\n', '\n').replace('\n\n', '')
                Skills.append(skill_text)
            else:
                Skills.append('')
        else:
            Skills.append('')
        
        Knowledge = soup.find('div', {'id': 'Knowledge'})
        if Knowledge is not None:
            ul = Knowledge.find('ul', {'class': 'list-unstyled m-0'})
            if ul is not None:
                knowledge_text = ul.text.replace('\nRelated occupations\n\n\n\n\n\n', '\n').replace('\nRelated occupations\n\n\n\n', '\n').replace('\n\n', '')
                Knowledges.append(knowledge_text)
            else:
                Knowledges.append('')
        else:
            Knowledges.append('')
        
        Technology_Skill = soup.find('div', {'id': 'TechnologySkills'})
        if Technology_Skill is not None:
            ul = Technology_Skill.find('ul', {'class': 'list-unstyled m-0'})
            if ul is not None:
                technology_skill_text = ul.text.replace('\nRelated occupations\n\n\n\n\n\n', '\n').replace('\nRelated occupations\n\n\n\n', '\n').replace('\n\n', '')
                Technology_Skills.append(technology_skill_text)
            else:
                Technology_Skills.append('')
        else:
            Technology_Skills.append('')
    
    total = {
        'Job_Titles': Job_Titles,
        'Job_Descriptions': Job_Descriptions,
        'Tasks': Tasks,
        'Skills': Skills,
        'Knowledges': Knowledges,
        'Technology_Skills': Technology_Skills
    }
    
    data = pd.DataFrame.from_dict(total, orient='index').transpose()
    data.to_excel(writer, sheet_name=sheet_name, index=False)

writer.save()


In [16]:
dataframes2 = pd.read_excel("total.xlsx", sheet_name=None)
for sheet_name, df in dataframes2.items():
    print("Sheet:", sheet_name)
    print(df)

Sheet: مشاغل مدیریت
                                           Job_Titles  \
0                       Chief Sustainability Officers   
1                     General and Operations Managers   
2                                Political Scientists   
3                 Advertising and Promotions Managers   
4             Energy Engineers, Except Wind and Solar   
5                                  Marketing Managers   
6                                      Sales Managers   
7                           Public Relations Managers   
8                         Regulatory Affairs Managers   
9           Computer and Information Systems Managers   
10                                 Financial Managers   
11                         Treasurers and Controllers   
12                                 Financial Managers   
13  First-Line Supervisors of Production and Opera...   
14                   Quality Control Systems Managers   
15                                     Range Managers   
16         

In [17]:
for i in sheet_names:
    dataframes2[i].dropna(inplace=True)

In [18]:
columns = (dataframes2[i].columns)
columns

Index(['Job_Titles', 'Job_Descriptions', 'Tasks', 'Skills', 'Knowledges',
       'Technology_Skills'],
      dtype='object')

In [19]:
translator = Translator()
def translate(row):
        if row[column] is not None:
            try:
                translation = translator.translate(row[column], src='en', dest='fa')
                if translation is not None and translation.text is not None:
                    return translation.text
            except Exception:
                pass
        return ''
for i in sheet_names:
    for column in columns:
        dataframes2[i][column] = dataframes2[i].apply(translate, axis=1)

In [23]:
dataframes2['مشاغل مدیریت']

,عنوان شغل,شرح شغل,وظایف,مهارت ها,دانش ها,مهارت های نرم افزاری
0,افسران ارشد پایداری,ارتباط و هماهنگی با مدیریت ، سهامداران ، مشتری...,نظارت و ارزیابی اثربخشی برنامه های پایداری.\nت...,تفکر انتقادی - استفاده از منطق و استدلال برای ...,زبان انگلیسی - دانش ساختار و محتوای زبان انگلی...,نرم افزار هوش تجاری و تجزیه و تحلیل داده ها - ...
1,مدیران عمومی و عملیات,برنامه ریزی ، مستقیم یا هماهنگی عملکرد سازمان ...,صورتهای مالی ، گزارش های فروش یا فعالیت یا سای...,گوش دادن فعال - توجه کامل به آنچه افراد دیگر م...,مدیریت و مدیریت - دانش اصول تجاری و مدیریتی در...,نرم افزار دسترسی - نرم افزار Citrix Cloud Comp...
2,دانشمندان سیاسی,منشأ ، توسعه و عملکرد سیستم های سیاسی را مطالع...,آموزش علوم سیاسی.\nدانش فعلی در مورد تصمیمات س...,درک مطلب-درک جملات و پاراگرافهای نوشتاری در اس...,قانون و دولت - دانش قوانین ، قوانین حقوقی ، رو...,نرم افزار تحلیلی یا علمی - آمار IBM SPSS ؛قاضی...
3,مدیران تبلیغات و تبلیغات,برنامه ها و برنامه های تبلیغاتی را مستقیم یا ه...,برای افزایش فروش محصولات یا خدمات ، کار با مشت...,گوش دادن فعال - توجه کامل به آنچه افراد دیگر م...,فروش و بازاریابی - دانش اصول و روشهای نمایش ، ...,نرم افزار تحلیلی یا علمی - نرم افزار تجسم داده...
4,مهندسان انرژی به جز باد و خورشیدی,طراحی ، توسعه یا ارزیابی پروژه ها یا برنامه ها...,استراتژی های صرفه جویی در مصرف انرژی را برای د...,درک مطلب-درک جملات و پاراگرافهای نوشتاری در اس...,مهندسی و فناوری - دانش کاربرد عملی علوم و فناو...,نرم افزار تحلیلی یا علمی - شرکت انرژی معماری E...
5,مدیران بازاریابی,برنامه ها و برنامه های بازاریابی را مستقیم یا ...,استراتژی بازاریابی را بر اساس دانش اهداف تأسیس...,یادگیری فعال-درک پیامدهای اطلاعات جدید برای حل...,فروش و بازاریابی - دانش اصول و روشهای نمایش ، ...,نرم افزار دسترسی - نرم افزار Citrix Cloud Comp...
6,مدیران فروش,برنامه ریزی ، مستقیم یا هماهنگی توزیع واقعی یا...,فعالیتهای مستقیم و هماهنگ شامل فروش محصولات تو...,اقناع - ترغیب دیگران برای تغییر نظر یا رفتار خ...,فروش و بازاریابی - دانش اصول و روشهای نمایش ، ...,نرم افزار حسابداری - حسابداری SAGE 50 ؛نرم افز...
8,مدیران امور نظارتی,برای اطمینان از رعایت مقررات و رویه های عملیات...,پاسخ به آژانس های نظارتی در مورد اطلاعات مربوط...,درک مطلب-درک جملات و پاراگرافهای نوشتاری در اس...,زبان انگلیسی - دانش ساختار و محتوای زبان انگلی...,نرم افزار تحلیلی یا علمی-Analyse-it ؛نرم افزار...
9,مدیران سیستم های رایانه ای و اطلاعاتی,فعالیت های مستقیم یا هماهنگی در زمینه هایی مان...,عملیات روزانه مستقیم بخش ، تجزیه و تحلیل گردش ...,تفکر انتقادی - استفاده از منطق و استدلال برای ...,رایانه ها و الکترونیک - دانش تابلوهای مدار ، پ...,نرم افزار دسترسی - نرم افزار Citrix Cloud Comp...
10,مدیران مالی,برنامه ریزی ، مستقیم یا هماهنگ حسابداری ، سرما...,برقراری و حفظ روابط با مشتریان فردی یا شغلی یا...,گوش دادن فعال - توجه کامل به آنچه افراد دیگر م...,خدمات مشتری و شخصی - دانش اصول و فرآیندهای ارا...,نرم افزار حسابداری - نرم افزار دریافتنی حساب ؛...


In [21]:
for i in sheet_names:
    dataframes2[i].rename(columns={'Job_Titles': 'عنوان شغل','Job_Descriptions': 'شرح شغل','Tasks': 'وظایف',
                              'Skills': 'مهارت ها','Knowledges': 'دانش ها','Technology_Skills': 'مهارت های نرم افزاری'}, inplace=True)

In [27]:
writer = pd.ExcelWriter('translated.xlsx')

# Write each dataframe to a separate sheet
for sheet_name, dataframe in dataframes2.items():
    dataframe.to_excel(writer, sheet_name=sheet_name, index=False)

# Save and close the ExcelWriter object
writer.save()